# **ECR - Engenharia**

### Importação de bibliotecas


In [3]:
import pandas as pd
import numpy as np
import re
import os
import pytesseract
from PIL import Image
from datetime import datetime
import requests
import json
from tabulate import tabulate
from openpyxl import Workbook


#### **Chaves de acesso**

In [4]:
APP_KEY = "596450736882"
APP_SECRET = "37a10edb420d8cd6c7afddcaaffcac05"

#### Enetendimento da Planilhas (Colunas/Linhas)

In [9]:
arquivo = 'APIS OMIE ECR.xlsx'

# Lista todas as abas da planilha
xls = pd.ExcelFile(arquivo)
print("Abas encontradas:", xls.sheet_names)

# Carrega a primeira aba (ou troque pelo nome desejado)
df = pd.read_excel(xls, sheet_name=0)

# Mostra as primeiras linhas para entender os dados
print("\nPrimeiras linhas:")
print(df.head())

# Informações gerais
print("\nInformações do DataFrame:")
print(df.info())

# Colunas disponíveis
print("\nColunas disponíveis:")
print(df.columns)

Abas encontradas: ['Planilha1']

Primeiras linhas:
                                               Geral  \
0  Alguns cadastros no Omie são compartilhados po...   
1       Clientes, Fornecedores, Transportadoras, etc   
2                         Clientes - Características   
3                                               Tags   
4                                           Projetos   

                                          Unnamed: 1 Unnamed: 2  \
0                                                NaN        NaN   
1  Cria/edita/consulta o cadastro de clientes, fo...         v1   
2    Cria/edita/consulta características de clientes         v1   
3  Cria/edita/consulta tags quem são usadas no ca...         v1   
4                       Cria/edita/consulta projetos         v1   

   DADO UTILIZADO NO GERENCIAL? Unnamed: 4  
0                           NaN        NaN  
1                           NaN         OK  
2                           NaN         OK  
3                           N

#### Filtro de endpoints

In [10]:
df.columns = ['Geral', 'Descricao', 'Versao', 'UsadoGerencial', 'Status']

# Filtrar linhas que possuem descrição e versão preenchidas
filtro_endpoints = df[ df['Versao'].notna()]

# Exibir os primeiros endpoints filtrados
filtro_endpoints.reset_index(drop=True)

,Geral,Descricao,Versao,UsadoGerencial,Status
0,"Clientes, Fornecedores, Transportadoras, etc","Cria/edita/consulta o cadastro de clientes, fo...",v1,NaN,OK
1,Clientes - Características,Cria/edita/consulta características de clientes,v1,NaN,OK
2,Tags,Cria/edita/consulta tags quem são usadas no ca...,v1,NaN,OK
3,Projetos,Cria/edita/consulta projetos,v1,NaN,OK
4,Empresas,Lista o cadastro da empresa,v1,NaN,OK
...,...,...,...,...,...
138,Etapas de Faturamento,Lista as etapas do faturamento,v1,NaN,OK
139,Tipo de utilização,Lista os Tipos de utilização,v1,NaN,OK
140,Classificação do Serviço,Lista as Classificações do Serviço,v1,NaN,OK
141,Documentos Fiscais,Listagem dos XMLs de Documentos Fiscais (NF-e/...,v1,NaN,OK


#### **Extração teste**

In [ ]:
endpoint = "clientes"
versao = "v1"

base_url = f"https://app.omie.com.br/api/v1/geral/{endpoint}/"


headers = {
    'Content-Type': 'application/json'
}


body = {
    'call': 'ListarClientes',
    'app_key': APP_KEY,
    'app_secret': APP_SECRET,
    'param': [
        {
            "pagina": 1,
            "registros_por_pagina": 50
        }
    ]
}


response = requests.post(base_url, headers=headers, data=json.dumps(body))

if response.status_code == 200:
    print("Dados extraídos com sucesso!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Erro na requisição: {response.status_code}")
    print(response.text)


Dados extraídos com sucesso!
{
  "pagina": 1,
  "total_de_paginas": 60,
  "registros": 50,
  "total_de_registros": 2963,
  "clientes_cadastro": [
    {
      "bairro": "",
      "bloquear_faturamento": "N",
      "cep": "",
      "cidade": "",
      "cidade_ibge": "",
      "cnpj_cpf": "000.000.000-00",
      "codigo_cliente_integracao": "_CLIENTE_CONSUMIDOR_",
      "codigo_cliente_omie": 671564307,
      "codigo_pais": "1058",
      "complemento": "",
      "dadosBancarios": {
        "agencia": "",
        "cChavePix": "",
        "codigo_banco": "",
        "conta_corrente": "",
        "doc_titular": "",
        "nome_titular": "",
        "transf_padrao": "N"
      },
      "endereco": "",
      "enderecoEntrega": {},
      "endereco_numero": "",
      "enviar_anexos": "N",
      "estado": "",
      "exterior": "N",
      "inativo": "N",
      "info": {
        "cImpAPI": "N",
        "dAlt": "20/12/2018",
        "dInc": "20/12/2018",
        "hAlt": "18:54:58",
        "hInc": 